# 0.匯入套件

In [1]:
import random
import requests

# 1.閒聊問答實作

## 1-1.自定義 Q&A

In [2]:
greating = "你是誰"
to_girl = [("請管好你的嘴","為什麼","因為我會隨時親你"),
           ("你單身多久了","一年多了","對不起，讓你等了這麼久"),
           ("我真的是太笨了","怎麼啦","除了喜歡你，其它什麼都做不好")]


def get_cutomize_ans(query):
    # 完全比對
    if query == greating:
        return "我是你的助理! 我可以陪你聊天喔~"

    # 部分比對
    elif "撩妹" in query:
        return str(random.choice(to_girl))

    # 模糊比對-1
    elif "台北" in query and "景點" in query:
        return "有101, 中正紀念堂還可以去爬象山喔!"
    
    elif any(i in query for i in "台北 台北市 台灣首都 天龍國".split()):
        if any(j in query for j in "手搖飲料店 珍奶店 飲料店".split()):
            url = "https://google.com/search?hl=zh-TW&q=台北市手搖飲料店&num=10"
            return url
        elif any(j in query for j in "名產 伴手禮 土產".split()):
            url = "https://google.com/search?hl=zh-TW&q=台北市名產&num=10"
            return url
        
        else:
            return None
            # return "我還沒辦法回覆你的問題喔!"
    else:
        return None
        # return "我還沒辦法回覆你的問題喔!"

In [3]:
queries = [
    "你是誰",
    "撩妹金句",
    "告訴我台北的知名景點有哪些",
    "景點有哪些",
    "台北市有幾間手搖飲料店",
    "台北市有幾間搖手店",
    "天龍國的伴手禮去哪裡買",
    "天龍人的伴手禮",   
]

for query in queries:
    answer = get_cutomize_ans(query)
    print(query,answer)
    print("===")

你是誰 我是你的助理! 我可以陪你聊天喔~
===
撩妹金句 ('請管好你的嘴', '為什麼', '因為我會隨時親你')
===
告訴我台北的知名景點有哪些 有101, 中正紀念堂還可以去爬象山喔!
===
景點有哪些 None
===
台北市有幾間手搖飲料店 https://google.com/search?hl=zh-TW&q=台北市手搖飲料店&num=10
===
台北市有幾間搖手店 None
===
天龍國的伴手禮去哪裡買 https://google.com/search?hl=zh-TW&q=台北市名產&num=10
===
天龍人的伴手禮 None
===


## 1-2. chatbot API

In [4]:
# 青雲客 API
def get_api_ans(query):
    URL = "http://api.qingyunke.com/api.php?key=free&appid=0&msg={}".format(query)
    headers = {'Content-Type': 'application/json'}
    res = requests.get(URL, headers=headers)
    return res.json()['content']

In [5]:
queries = [
#    "你不是已經在家了嗎",
#     "重點是什麼呢",
     "我最近睡不著",
#     "你聽過鬼滅之刃嗎",
#     "一起去玩啦",
#     "風大雨大太陽大",
#     "我好想吃冰喔",
#     "呵呵，如果是這樣就沒救囉",
#     "佛祖會生氣喔",
#     "你會溜直排輪嗎"
]

for query in queries:
    answer = get_api_ans(query)
    print(query,answer)
    print("===")

我最近睡不著 菲菲不需要睡觉的，精神得很呢
===


# 2.知識問答實作

In [6]:
import jieba
import jieba.posseg
import urllib.parse
import requests, re
from bs4 import BeautifulSoup


# 進行斷詞
def jieba_tokenizer(query):
    word_pos = jieba.posseg.cut(query)
    return [(word,pos) for word, pos in word_pos]

# 取得 html   
def get_html(query, num):
    query = urllib.parse.quote_plus(query)
    url = 'https://google.com/search?hl=zh-TW&q=' + query + "&num=" + str(num)
    try:
        response = requests.get(url)
        return BeautifulSoup(response.text, 'lxml')
    except:
        return None
    
# 取得搜尋結果的標題
def get_title(text):
    title = re.findall(r'<h3 class="zBAuLc"><div class="BNeawe vvjwJb AP7Wnd">(?P<content>.*?)</div></h3>', str(text), re.DOTALL)
    if title != []:
        title = title[0].replace('\n', '')
    return title

# 取得結果摘要
def get_content(text):
    content = re.findall(r'<div class="kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd"><div><div><div class="BNeawe s3v9rd AP7Wnd">(?P<content>.*?)</div></div></div></div></div></div></div>', str(text), re.DOTALL)
    if content != []:
        content = content[0].replace('\n', '')
        content = re.sub(u'<span class="r0bn4c rQMQod">.*?"r0bn4c rQMQod"> · </span>', "", content)
    return content

def get_encyclopedia_ans(query):
    token = jieba_tokenizer(query)
    html = get_html(query, 1)

    candidate = html.select('div.ZINbbc.xpd.O9g5cc.uUPGi')
    answer = get_content(candidate)
    return answer

In [7]:
queries = [
    "馬英九是誰", 
    "介紹吳宗憲", 
    "拜登的生平", 
    "含羞草的原理", 
    "黑死病發生的時間", 
    "小狗的智商"
]

for query in queries:
    answer = get_encyclopedia_ans(query)
    print(query,answer)
    print("===")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\aband\AppData\Local\Temp\jieba.cache
Loading model cost 1.516 seconds.
Prefix dict has been built successfully.


馬英九是誰 2012年順利连任中华民国总统。2015年11月7日，馬英九在新加坡與中國共產黨中央委員會總書記習近平進行兩岸領導人會面，是兩岸隔海分治 ...副市長: <span class="BNeawe"><a href="/url?q=https://zh.wikipedia.org/wiki/%25E7%2599%25BD%25E7%25A7%2580%25E9%259B%2584&amp;sa=U&amp;ved=2ahUKEwjy3J-9yrHwAhWKy4sBHaYMCUMQs2YwC3oECAAQBA&amp;usg=AOvVaw2J8FJd3G6Q9W42VDgKWiwY"><span class="XLloXe AP7Wnd">白秀雄</span></a></span><span class="r0bn4c rQMQod">; </span><span class="BNeawe"><a href="/url?q=https://zh.wikipedia.org/wiki/%25E9%2587%2591%25E6%25BA%25A5%25E8%2581%25B0&amp;sa=U&amp;ved=2ahUKEwjy3J-9yrHwAhWKy4sBHaYMCUMQs2YwC3oECAAQBQ&amp;usg=AOvVaw21u8oY_d-uxnDJ4a7aJYD9"><span class="XLloXe AP7Wnd">金溥聰</span></a></span>副总统: <span class="r0bn4c rQMQod">蕭萬長（第12任）; 吳敦義（第13任）</span>居住地: <span class="r0bn4c rQMQod">中華民國臺北市文山區</span>服役时间: <span class="r0bn4c rQMQod">1972年－1974年</span>
===
介紹吳宗憲 吳宗憲（英語：Jacky Wu，1962年9月26日－），台灣男藝人、主持人、歌手、 唱片製作人、 ... 2006年左右，好友沈玉琳因工作結識從事光源生意的「台灣光屋照明事業集團」老闆，並介紹吳宗憲當代言人，因而引發吳對照明事業的興趣，他開始 ...<span class="BNeawe"><a href="/url?q=https://zh.wik

# 3.聊天機器人answer selection

In [8]:
# jieba 詞性參考 => http://blog.pulipuli.info/2017/11/fasttag-identify-part-of-speech-in.html

def query_classifier(token):
    '''
    return
     True => samll talk
     False => encyclopedia
    '''
    count_nouns, count_verbs, count_advs, count_adjs = 0, 0, 0, 0

    token_len = len(token)
    for i, token in enumerate(token):
        word, pos = token

        if pos == "x": # 標點符號
            token_len -= 1
                      
        if any(pos == p for p in "n ng nr nrfg nrt ns nt nz".split(" ")): # 名詞類
            count_nouns += 1
        elif any(pos == p for p in "v vg vd vi vn vq".split(" ")): # 動詞類
            count_verbs += 1

    
    # 若完全沒比對出名詞 可能為口語短句 應忽略
    if count_nouns == 0:
        return True
     # 若動詞超過兩個 可能為非口語短句 不應忽略
    elif count_verbs >= 2:
        return False
    else:
        return False
    
def get_response(query):
    token = jieba_tokenizer(query)
    res = query_classifier(token)
    
    cutomize_ans = get_cutomize_ans(query)
    # 先判斷是否為自定義語句
    if cutomize_ans != None:
        return cutomize_ans
    else:
        # 閒聊問句
        if res:
            return get_api_ans(query)
        # 知識問句
        else:
            return get_encyclopedia_ans(query)   

In [9]:
queries = [
    "蔡英文是誰", 
    "介紹陳零九", 
    "柴可夫斯基的生平", 
    "蘋果為什麼會變紅", 
    "為什麼這群人是網紅霸主", 
    "熱氣球的原理",   
    
#     "馬英九是誰", 
#     "介紹吳宗憲", 
#     "拜登的生平", 
#     "含羞草的原理", 
#     "黑死病發生的時間", 
#     "海豚的智商",  
    
    "你不是已經在家了嗎",
    "重點是什麼呢", # @@
    "我最近睡不著",
    "你聽過鬼滅之刃嗎",  # @@
    "一起去玩啦",
    "風大雨大太陽大",
    "我好想吃冰喔",  # @@
    "呵呵，如果是這樣就沒救囉",
    "佛祖會生氣喔",  # @@
    "你會溜直排輪嗎",
    
#     "你是誰",
#     "撩妹金句",
#     "告訴我台北的知名景點有哪些",
#     "景點有哪些",
#     "台北市有幾間手搖飲料店",
#     "台北市有幾間搖手店",
#     "天龍國的伴手禮去哪裡買",
#     "天龍人的伴手禮"
]
import time
for query in queries:
    response = get_response(query)
    print("Q:",query)
    print("A:",response)
    print("===")
    time.sleep(5)

Q: 蔡英文是誰
A: 蔡英文（排灣語：Tjuku；英語：Tsai Ing-Wen；1956年8月31日－），中華民國民主進步黨籍政治人物，現任中華民國總統兼民主進步黨主席，亦為法律學者與 ...<span class="BNeawe"><a href="/url?q=https://zh.wikipedia.org/wiki/%25E8%2594%25A1%25E6%25BD%2594%25E7%2594%259F&amp;sa=U&amp;ved=2ahUKEwiso4DAyrHwAhWUOZQKHcXPBjAQ0gIwC3oECAAQAg&amp;usg=AOvVaw3WOCo1uwofpBrzGT15P8-S"><span class="XLloXe AP7Wnd">蔡潔生</span></a></span> · <span class="BNeawe"><a href="/url?q=https://zh.wikipedia.org/wiki/%25E9%25A6%25AC%25E8%258B%25B1%25E4%25B9%259D&amp;sa=U&amp;ved=2ahUKEwiso4DAyrHwAhWUOZQKHcXPBjAQ0gIwC3oECAAQAw&amp;usg=AOvVaw3aPu6tS0LGwI1UR5b6KLdV"><span class="XLloXe AP7Wnd">馬英九</span></a></span> · <span class="BNeawe"><a href="/url?q=https://zh.wikipedia.org/wiki/%25E8%2594%25A1%25E8%258B%25B1%25E6%2596%2587%25E5%25AD%25B8%25E4%25BD%258D%25E9%2596%2580&amp;sa=U&amp;ved=2ahUKEwiso4DAyrHwAhWUOZQKHcXPBjAQ0gIwC3oECAAQBA&amp;usg=AOvVaw3Lb4AWvlDY45fbhOpH6TJD"><span class="XLloXe AP7Wnd">蔡英文學位門[编辑]</span></a></span> · <span class="BNeawe"><a href="/url?q=https://z